In [1]:
"""
GIRASOUL BUSINESS DASHBOARD - COMPLETE DATA IMPORT SYSTEM
Final comprehensive script combining all successful solutions

WHAT THIS SCRIPT DOES:
1. Imports inventory data from inventory_data.xlsx
2. Imports sold data from sold_data.xlsx  
3. Creates proper expense transactions for ALL purchased items (inventory + COGS)
4. Creates income transactions for all sold items
5. Handles SKU conflicts with priority system
6. Maps item categories correctly
7. Validates complete business logic

BUSINESS LOGIC IMPLEMENTED:
- Every purchased item → Expense transaction (Cost of Goods Sold)
- Items still in inventory → Expense only
- Items sold → Expense (COGS) + Income (sale)
- Proper financial tracking for profit/loss calculations

FILES REQUIRED:
- inventory_data.xlsx (current inventory items)
- sold_data.xlsx (sold items with COGS data)

EXPECTED RESULTS:
- ~167 inventory items with expense transactions
- ~312 sold items with both expense (COGS) and income transactions  
- Total expenses: ~$3,456.69 ($1,199.94 inventory + $2,256.75 COGS)
- Complete business financial tracking
"""

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, date
import traceback

# =============================================================================
# CONFIGURATION AND SETUP
# =============================================================================

# Database configuration
DATABASE_PATH = '../business.db'

# File paths
INVENTORY_FILE = 'inventory_data.xlsx'
SOLD_FILE = 'sold_data.xlsx'

# Category mapping for item types
CATEGORY_MAPPING = {
    # Clothing items
    'Dress': 'Clothing', 'Shirt': 'Clothing', 'Top': 'Clothing', 'Blouse': 'Clothing',
    'Sweater': 'Clothing', 'Jacket': 'Clothing', 'Coat': 'Clothing', 'Blazer': 'Clothing',
    'Cardigan': 'Clothing', 'Tank': 'Clothing', 'Tee': 'Clothing', 'Tunic': 'Clothing',
    'Romper': 'Clothing', 'Jumpsuit': 'Clothing',
    
    # Footwear
    'Boots': 'Footwear', 'Shoes': 'Footwear', 'Heels': 'Footwear', 
    'Sandals': 'Footwear', 'Sneakers': 'Footwear', 'Flats': 'Footwear',
    
    # Bottoms
    'Jeans': 'Bottoms', 'Pants': 'Bottoms', 'Shorts': 'Bottoms', 
    'Skirt': 'Bottoms', 'Leggings': 'Bottoms',
    
    # Accessories
    'Bag': 'Accessories', 'Purse': 'Accessories', 'Belt': 'Accessories',
    'Scarf': 'Accessories', 'Hat': 'Accessories', 'Jewelry': 'Accessories',
    'Watch': 'Accessories', 'Sunglasses': 'Accessories'
}

# Platform mapping for sold items
PLATFORM_MAPPING = {
    'IG': 'Instagram',
    'FB': 'Facebook', 
    'Poshmark': 'Poshmark',
    'F&F': 'Friends & Family'
}

# Global statistics tracking
IMPORT_STATS = {
    'inventory_imported': 0,
    'sold_imported': 0,
    'inventory_expenses_created': 0,
    'cogs_expenses_created': 0,
    'income_transactions_created': 0,
    'skus_resolved': 0,
    'errors': 0
}

print("🚀 GIRASOUL COMPLETE DATA IMPORT SYSTEM")
print("=" * 60)
print("📋 Comprehensive solution for Excel data import with complete business logic")
print("=" * 60)


🚀 GIRASOUL COMPLETE DATA IMPORT SYSTEM
📋 Comprehensive solution for Excel data import with complete business logic


In [2]:
# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def connect_database():
    """Connect to SQLite database"""
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        print(f"✅ Connected to database: {DATABASE_PATH}")
        return conn, cursor
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None, None

def clean_and_validate_data(value, data_type='text'):
    """Clean and validate data values"""
    if pd.isna(value) or value == '':
        return None if data_type == 'text' else 0.0 if data_type == 'currency' else None
    
    if data_type == 'text':
        return str(value).strip()
    elif data_type == 'currency':
        try:
            return float(str(value).replace('$', '').replace(',', ''))
        except:
            return 0.0
    elif data_type == 'date':
        if isinstance(value, str):
            try:
                return pd.to_datetime(value).date()
            except:
                return date.today()
        elif isinstance(value, datetime):
            return value.date()
        elif isinstance(value, date):
            return value
        else:
            return date.today()
    
    return value

def get_database_state(cursor):
    """Get current database state for reporting"""
    try:
        tables = ['business_inventory', 'business_sold', 'business_transactions']
        state = {}
        
        for table in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            state[table] = cursor.fetchone()[0]
        
        return state
    except Exception as e:
        print(f"⚠️ Error getting database state: {e}")
        return {}

In [3]:
# =============================================================================
# DATA LOADING AND ANALYSIS
# =============================================================================

def load_and_analyze_files():
    """Load both Excel files and analyze data structure"""
    
    print("\n📊 LOADING AND ANALYZING EXCEL FILES")
    print("=" * 45)
    
    try:
        # Load inventory data
        inventory_df = pd.read_excel(INVENTORY_FILE)
        print(f"✅ Loaded {INVENTORY_FILE}: {len(inventory_df)} rows")
        
        # Load sold data
        sold_df = pd.read_excel(SOLD_FILE)
        print(f"✅ Loaded {SOLD_FILE}: {len(sold_df)} rows")
        
        # Analyze SKU conflicts
        inventory_skus = set(inventory_df['SKU'].dropna().astype(str))
        sold_skus = set(sold_df['SKU'].dropna().astype(str))
        sku_conflicts = inventory_skus.intersection(sold_skus)
        
        print(f"\n🔍 SKU Analysis:")
        print(f"  Inventory SKUs: {len(inventory_skus)}")
        print(f"  Sold SKUs: {len(sold_skus)}")
        print(f"  Conflicting SKUs: {len(sku_conflicts)}")
        
        # Calculate totals
        inventory_cost_total = inventory_df['Cost of Item'].sum() if 'Cost of Item' in inventory_df.columns else 0
        sold_cost_total = sold_df['Cost'].sum() if 'Cost' in sold_df.columns else 0
        total_expected_expenses = inventory_cost_total + sold_cost_total
        
        print(f"\n💰 Expected Financial Totals:")
        print(f"  Inventory costs: ${inventory_cost_total:.2f}")
        print(f"  Sold item COGS: ${sold_cost_total:.2f}")
        print(f"  Total expected expenses: ${total_expected_expenses:.2f}")
        
        IMPORT_STATS['skus_resolved'] = len(sku_conflicts)
        
        return {
            'inventory_df': inventory_df,
            'sold_df': sold_df,
            'sku_conflicts': sku_conflicts,
            'inventory_cost_total': inventory_cost_total,
            'sold_cost_total': sold_cost_total,
            'expected_expenses': total_expected_expenses
        }
        
    except Exception as e:
        print(f"❌ Error loading files: {e}")
        traceback.print_exc()
        return None

In [4]:

# =============================================================================
# DATA TRANSFORMATION
# =============================================================================

def transform_inventory_data(inventory_df, sku_conflicts):
    """Transform inventory data for database import"""
    
    print("\n🔄 TRANSFORMING INVENTORY DATA")
    print("=" * 35)
    
    transformed_inventory = []
    skipped_conflicts = 0
    max_sku = 1000  # Start new SKUs from 1000 to avoid conflicts
    
    for index, row in inventory_df.iterrows():
        try:
            original_sku = clean_and_validate_data(row['SKU'], 'text')
            
            # Skip if SKU exists in sold data (sold items take priority)
            if original_sku in sku_conflicts:
                skipped_conflicts += 1
                continue
            
            # Use original SKU or generate new one
            sku = original_sku if original_sku else str(max_sku)
            max_sku += 1
            
            brand = clean_and_validate_data(row['Brand'], 'text') or 'Unknown'
            item_type = clean_and_validate_data(row['Item Type'], 'text') or 'Item'
            
            # Map category
            mapped_category = CATEGORY_MAPPING.get(item_type, 'Other')
            
            # Create name/description
            name = f"{brand} {item_type}".strip()
            
            inventory_item = {
                'sku': sku,
                'name': name,
                'description': name,
                'category': mapped_category,
                'cost_of_item': clean_and_validate_data(row['Cost of Item'], 'currency'),
                'selling_price': clean_and_validate_data(row['Selling Price'], 'currency'),
                'w_tax_price': clean_and_validate_data(row['W Tax'], 'currency'),
                'listing_status': 'inventory',
                'location': 'Inventory',
                'size': clean_and_validate_data(row['Size'], 'text'),
                'condition': clean_and_validate_data(row['Condition'], 'text'),
                'brand': brand,
                'drop_field': clean_and_validate_data(row['Drop'], 'text'),
                'date_added': clean_and_validate_data(row['Date'], 'date')
            }
            
            transformed_inventory.append(inventory_item)
            
        except Exception as e:
            print(f"⚠️ Error processing inventory row {index}: {e}")
            IMPORT_STATS['errors'] += 1
            continue
    
    print(f"✅ Transformed {len(transformed_inventory)} inventory items")
    print(f"⏭️ Skipped {skipped_conflicts} items due to SKU conflicts")
    
    return transformed_inventory

def transform_sold_data(sold_df):
    """Transform sold data for database import"""
    
    print("\n🔄 TRANSFORMING SOLD DATA")
    print("=" * 30)
    
    transformed_sold = []
    
    for index, row in sold_df.iterrows():
        try:
            sku = clean_and_validate_data(row['SKU'], 'text')
            brand = clean_and_validate_data(row['Brand'], 'text') or 'Unknown'
            item_type = clean_and_validate_data(row['Type'], 'text') or 'Item'
            
            # Map category
            mapped_category = CATEGORY_MAPPING.get(item_type, 'Other')
            
            # Create name/description
            name = f"{brand} {item_type}".strip()
            
            # Map platform
            status = clean_and_validate_data(row['Status'], 'text') or 'Other'
            platform = PLATFORM_MAPPING.get(status, status)
            
            sold_item = {
                'sku': sku,
                'name': name,
                'description': name,
                'category': mapped_category,
                'cost_of_item': clean_and_validate_data(row['Cost'], 'currency'),
                'selling_price': clean_and_validate_data(row['Price'], 'currency'),
                'sold_price': clean_and_validate_data(row['Sold'], 'currency'),
                'w_tax_price': clean_and_validate_data(row['W Tax'], 'currency'),
                'sold_date': clean_and_validate_data(row['Sold Date'], 'date'),
                'platform': platform,
                'location': 'Inventory',
                'size': clean_and_validate_data(row['Size'], 'text'),
                'condition': clean_and_validate_data(row['Condition'], 'text'),
                'brand': brand,
                'drop_field': clean_and_validate_data(row['Drop'], 'text')
            }
            
            transformed_sold.append(sold_item)
            
        except Exception as e:
            print(f"⚠️ Error processing sold row {index}: {e}")
            IMPORT_STATS['errors'] += 1
            continue
    
    print(f"✅ Transformed {len(transformed_sold)} sold items")
    
    return transformed_sold

In [5]:
# =============================================================================
# DATABASE IMPORT FUNCTIONS
# =============================================================================

def import_inventory_data(cursor, inventory_items):
    """Import inventory data and create expense transactions"""
    
    print("\n📦 IMPORTING INVENTORY DATA")
    print("=" * 35)
    
    success_count = 0
    transaction_count = 0
    
    try:
        cursor.execute("BEGIN TRANSACTION")
        
        for item in inventory_items:
            try:
                # Insert inventory item
                inventory_sql = """
                INSERT INTO business_inventory (
                    sku, name, description, category, cost_of_item, selling_price,
                    w_tax_price, listing_status, location, size, condition, brand, drop_field
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """
                
                cursor.execute(inventory_sql, (
                    item['sku'], item['name'], item['description'], item['category'],
                    item['cost_of_item'], item['selling_price'], item['w_tax_price'],
                    item['listing_status'], item['location'], item['size'],
                    item['condition'], item['brand'], item['drop_field']
                ))
                
                success_count += 1
                
                # Create expense transaction if cost > 0
                if item['cost_of_item'] and item['cost_of_item'] > 0:
                    transaction_sql = """
                    INSERT INTO business_transactions (
                        date, description, amount, category, sub_category,
                        transaction_type, account_name, notes
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    """
                    
                    cursor.execute(transaction_sql, (
                        item['date_added'],
                        f"Inventory Purchase - {item['name']}",
                        item['cost_of_item'],
                        'Cost of Goods Sold',
                        'Inventory Purchase',
                        'Expense',
                        'Business Account',
                        f"SKU: {item['sku']}"
                    ))
                    
                    transaction_count += 1
                
            except Exception as e:
                print(f"❌ Error importing inventory SKU {item['sku']}: {e}")
                IMPORT_STATS['errors'] += 1
                continue
        
        cursor.execute("COMMIT")
        
        print(f"✅ Inventory import complete:")
        print(f"  Items imported: {success_count}")
        print(f"  Expense transactions: {transaction_count}")
        
        IMPORT_STATS['inventory_imported'] = success_count
        IMPORT_STATS['inventory_expenses_created'] = transaction_count
        
        return success_count, transaction_count
        
    except Exception as e:
        cursor.execute("ROLLBACK")
        print(f"❌ Inventory import failed: {e}")
        return 0, 0

def import_sold_data(cursor, sold_items):
    """Import sold data and create both COGS expense and income transactions"""
    
    print("\n💰 IMPORTING SOLD DATA")
    print("=" * 30)
    
    success_count = 0
    cogs_transactions = 0
    income_transactions = 0
    
    try:
        cursor.execute("BEGIN TRANSACTION")
        
        for item in sold_items:
            try:
                # Insert sold item
                sold_sql = """
                INSERT INTO business_sold (
                    sku, name, description, category, cost_of_item, selling_price,
                    sold_price, w_tax_price, sold_date, platform, location,
                    size, condition, brand, drop_field
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """
                
                cursor.execute(sold_sql, (
                    item['sku'], item['name'], item['description'], item['category'],
                    item['cost_of_item'], item['selling_price'], item['sold_price'],
                    item['w_tax_price'], item['sold_date'], item['platform'],
                    item['location'], item['size'], item['condition'],
                    item['brand'], item['drop_field']
                ))
                
                success_count += 1
                
                # Create COGS expense transaction
                if item['cost_of_item'] and item['cost_of_item'] > 0:
                    transaction_sql = """
                    INSERT INTO business_transactions (
                        date, description, amount, category, sub_category,
                        transaction_type, account_name, notes
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    """
                    
                    cursor.execute(transaction_sql, (
                        item['sold_date'],
                        f"Cost of Goods Sold - {item['name']}",
                        item['cost_of_item'],
                        'Cost of Goods Sold',
                        'Sold Inventory',
                        'Expense',
                        'Business Account',
                        f"SKU: {item['sku']}, Platform: {item['platform']}, COGS for sold item"
                    ))
                    
                    cogs_transactions += 1
                
                # Create income transaction for sale
                if item['sold_price'] and item['sold_price'] > 0:
                    cursor.execute(transaction_sql, (
                        item['sold_date'],
                        f"Sale - {item['name']}",
                        item['sold_price'],
                        'Sales Revenue',
                        'Product Sales',
                        'Income',
                        'Business Account',
                        f"SKU: {item['sku']}, Platform: {item['platform']}"
                    ))
                    
                    income_transactions += 1
                
            except Exception as e:
                print(f"❌ Error importing sold SKU {item['sku']}: {e}")
                IMPORT_STATS['errors'] += 1
                continue
        
        cursor.execute("COMMIT")
        
        print(f"✅ Sold data import complete:")
        print(f"  Items imported: {success_count}")
        print(f"  COGS expense transactions: {cogs_transactions}")
        print(f"  Income transactions: {income_transactions}")
        
        IMPORT_STATS['sold_imported'] = success_count
        IMPORT_STATS['cogs_expenses_created'] = cogs_transactions
        IMPORT_STATS['income_transactions_created'] = income_transactions
        
        return success_count, cogs_transactions, income_transactions
        
    except Exception as e:
        cursor.execute("ROLLBACK")
        print(f"❌ Sold data import failed: {e}")
        return 0, 0, 0

In [6]:
# =============================================================================
# VALIDATION AND REPORTING
# =============================================================================

def validate_import_results(cursor, expected_data):
    """Validate the complete import results"""
    
    print("\n✅ VALIDATING IMPORT RESULTS")
    print("=" * 35)
    
    # Check record counts
    final_state = get_database_state(cursor)
    
    print(f"📊 Final Database State:")
    for table, count in final_state.items():
        print(f"  {table}: {count} records")
    
    # Check expense totals
    cursor.execute("""
        SELECT 
            COUNT(*) as expense_count,
            SUM(amount) as total_expenses
        FROM business_transactions 
        WHERE transaction_type = 'Expense'
    """)
    
    expense_count, total_expenses = cursor.fetchone()
    
    # Check expense breakdown
    cursor.execute("""
        SELECT 
            sub_category,
            COUNT(*) as count,
            SUM(amount) as amount
        FROM business_transactions 
        WHERE transaction_type = 'Expense'
        AND category = 'Cost of Goods Sold'
        GROUP BY sub_category
    """)
    
    expense_breakdown = cursor.fetchall()
    
    print(f"\n💳 Expense Analysis:")
    print(f"  Total expense transactions: {expense_count}")
    print(f"  Total expense amount: ${total_expenses or 0:.2f}")
    print(f"  Expected amount: ${expected_data['expected_expenses']:.2f}")
    
    print(f"\n📦 Expense Breakdown:")
    for sub_cat, count, amount in expense_breakdown:
        print(f"  {sub_cat or 'Other'}: {count} transactions, ${amount or 0:.2f}")
    
    # Check income totals
    cursor.execute("""
        SELECT 
            COUNT(*) as income_count,
            SUM(amount) as total_income
        FROM business_transactions 
        WHERE transaction_type = 'Income'
    """)
    
    income_count, total_income = cursor.fetchone()
    
    print(f"\n💰 Income Analysis:")
    print(f"  Income transactions: {income_count}")
    print(f"  Total income: ${total_income or 0:.2f}")
    
    # Calculate success metrics
    expense_accuracy = abs((total_expenses or 0) - expected_data['expected_expenses'])
    success_rate = ((IMPORT_STATS['inventory_imported'] + IMPORT_STATS['sold_imported']) / 
                   (IMPORT_STATS['inventory_imported'] + IMPORT_STATS['sold_imported'] + IMPORT_STATS['errors'])) * 100
    
    print(f"\n🎯 Success Metrics:")
    print(f"  Expense accuracy: ${expense_accuracy:.2f} difference")
    print(f"  Import success rate: {success_rate:.1f}%")
    
    if expense_accuracy < 10 and success_rate > 95:
        print(f"✅ IMPORT SUCCESSFUL - All business logic implemented correctly!")
        return True
    else:
        print(f"⚠️ Some issues detected - review results")
        return False

def generate_final_report():
    """Generate comprehensive final report"""
    
    print("\n📋 FINAL IMPORT REPORT")
    print("=" * 30)
    print(f"📅 Import Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    print(f"\n📊 Import Statistics:")
    for key, value in IMPORT_STATS.items():
        formatted_key = key.replace('_', ' ').title()
        print(f"  {formatted_key}: {value}")
    
    total_transactions = (IMPORT_STATS['inventory_expenses_created'] + 
                         IMPORT_STATS['cogs_expenses_created'] + 
                         IMPORT_STATS['income_transactions_created'])
    
    print(f"\n💳 Transaction Summary:")
    print(f"  Total transactions created: {total_transactions}")
    print(f"  Expense transactions: {IMPORT_STATS['inventory_expenses_created'] + IMPORT_STATS['cogs_expenses_created']}")
    print(f"  Income transactions: {IMPORT_STATS['income_transactions_created']}")
    
    print(f"\n✅ Business Logic Implemented:")
    print(f"  ✅ Complete COGS tracking for all purchased items")
    print(f"  ✅ Proper expense transactions for inventory and sold items")
    print(f"  ✅ Income transactions for all sales")
    print(f"  ✅ SKU conflict resolution with priority system")
    print(f"  ✅ Category mapping for all item types")
    
    print(f"\n🎯 Financial Dashboard Ready:")
    print(f"  Dashboard will show accurate expense/income totals")
    print(f"  Profit/loss calculations will be correct")
    print(f"  All business operations properly tracked")

In [7]:
# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

def execute_complete_import():
    """Execute the complete data import process"""
    
    print("\n🚀 EXECUTING COMPLETE IMPORT PROCESS")
    print("=" * 45)
    
    # Connect to database
    conn, cursor = connect_database()
    if not conn:
        return False
    
    try:
        # Load and analyze files
        file_data = load_and_analyze_files()
        if not file_data:
            return False
        
        # Transform data
        inventory_items = transform_inventory_data(file_data['inventory_df'], file_data['sku_conflicts'])
        sold_items = transform_sold_data(file_data['sold_df'])
        
        # Import data
        inv_success, inv_transactions = import_inventory_data(cursor, inventory_items)
        sold_success, cogs_transactions, income_transactions = import_sold_data(cursor, sold_items)
        
        # Validate results
        validation_success = validate_import_results(cursor, file_data)
        
        # Generate final report
        generate_final_report()
        
        print(f"\n🎉 COMPLETE IMPORT FINISHED!")
        print(f"📊 Summary: {inv_success + sold_success} items imported")
        print(f"💳 Transactions: {inv_transactions + cogs_transactions + income_transactions} created")
        print(f"✅ Business logic: Complete and accurate")
        
        return validation_success
        
    except Exception as e:
        print(f"❌ Import process failed: {e}")
        traceback.print_exc()
        return False
        
    finally:
        if conn:
            conn.close()
            print(f"\n✅ Database connection closed")

In [8]:
# =============================================================================
# SCRIPT EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("📋 READY TO EXECUTE COMPLETE IMPORT")
    print("=" * 60)
    print("This script will:")
    print("1. Import all inventory data with expense transactions")
    print("2. Import all sold data with COGS expenses and income transactions") 
    print("3. Resolve SKU conflicts automatically")
    print("4. Validate complete business logic")
    print("5. Generate comprehensive report")
    print("")
    print("FILES REQUIRED:")
    print(f"  - {INVENTORY_FILE}")
    print(f"  - {SOLD_FILE}")
    print("")
    print("Uncomment the line below to execute:")
    print("# import_success = execute_complete_import()")
    print("=" * 60)
    
    # Uncomment to run the complete import
    import_success = execute_complete_import()


📋 READY TO EXECUTE COMPLETE IMPORT
This script will:
1. Import all inventory data with expense transactions
2. Import all sold data with COGS expenses and income transactions
3. Resolve SKU conflicts automatically
4. Validate complete business logic
5. Generate comprehensive report

FILES REQUIRED:
  - inventory_data.xlsx
  - sold_data.xlsx

Uncomment the line below to execute:
# import_success = execute_complete_import()

🚀 EXECUTING COMPLETE IMPORT PROCESS
✅ Connected to database: ../business.db

📊 LOADING AND ANALYZING EXCEL FILES
✅ Loaded inventory_data.xlsx: 167 rows
✅ Loaded sold_data.xlsx: 312 rows

🔍 SKU Analysis:
  Inventory SKUs: 167
  Sold SKUs: 312
  Conflicting SKUs: 0

💰 Expected Financial Totals:
  Inventory costs: $1199.94
  Sold item COGS: $2256.75
  Total expected expenses: $3456.69

🔄 TRANSFORMING INVENTORY DATA
✅ Transformed 167 inventory items
⏭️ Skipped 0 items due to SKU conflicts

🔄 TRANSFORMING SOLD DATA
✅ Transformed 312 sold items

📦 IMPORTING INVENTORY DATA